0. Cargar librerías

In [3]:
from google.colab import drive
drive.mount("/content/drive")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Google minería de datos
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import roc_curve
#Métricas para evaluar los modelos
from sklearn.metrics import confusion_matrix
#Graficador para hacer dashboards
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

ValueError: mount failed

**KNN COMO MODELO DE CLASIFICACIÓN Y AGRUPAMIENTO**

In [ ]:
XDB=pd.read_excel('/content/drive/MyDrive/AnaliticaNegocios(01-2024)/salaries-_2.xlsx', sheet_name=1)
XDB.head()
XDB=XDB[['work_year','salary','job_title', 'salary_in_usd', 'company_size']]
XDB=XDB.dropna()
XDB.corr()
dfcorrel=pd.DataFrame(XDB.corr())
dfcorrel.to_excel('InformeCorrel.xlsx')

In [ ]:
#creamos el dashboard
sns.pairplot(XDB, hue='job_title', size=4, vars=['work_year','salary', 'salary_in_usd','company_size'])

In [ ]:
#datos de entrada - datos de salida
XD=np.array(XDB[['work_year','salary', 'salary_in_usd','company_size']])
yd=np.array(XDB[['job_title']])

mknn=KNeighborsClassifier(n_neighbors=10)
mknn.fit(XD,yd)
ydp=mknn.predict(XD)

print("La precision en el pronostico es:", metrics.accuracy_score(yd,ydp))

In [ ]:
#Matriz de confusión
cm=confusion_matrix(yd,ydp)
print(cm)
df=pd.DataFrame(cm)
df.to_excel('MatrizConfusion.xlsx')

In [ ]:
#Metricas sobre el comportamiento en general de modelo
VP=cm[0,0];FN=cm[0,1]
FP=cm[1,0];VN=cm[1,1]
print("La exactitud es:", (VP+VN)/(VP+FN+FP+VN))
#Comportamiento tasa de error
print("La Tasa de Error es:", (FP+FN)/(VP+FN+FP+VN))
#COEFICIENTE DE SENSIBILIDAD DEL MODELO
print("El coeficiente de sensibilidad",VP/(VP+FN))
#COEFICIENTE DE ESPECIFICIDAD
print("El coeficiente de especificidad", VN/(FP+VN))
#LA CALIDAD DE LOS DATOS PARA EL PRONOSTICO POSITIVO ES
print("Pronostico Positivo:", VP/(VP+FP))
#LA CALIDAD DE LOS DATOS PARA EL PRONOSTICO NEGATIVO ES
print("Pronostico Negativo",VN/(FN+VN))

- Evaluamos a un individuo

In [ ]:
XDi=np.array([(2024,12000,87664,2)])
ydi=mknn.predict(XDi)
print("La decision frente al titulo del trabajo es:", ydi)

plt.scatter(XD[:,3],XD[:,1],c=yd, s=50, cmap='inferno')  #c es como se colocan los datos, s tamaño de punto, cmap el color
plt.scatter(XDi[:,3],XDi[:,1],marker='o',s=500,facecolor='green')
plt.title('Company Size vs Salary')
plt.show()

- Madaline de pronostico

In [ ]:
yd2=np.array(XDB[['job_title']])
yd2c=(yd2-np.mean(yd2))/np.std(yd2) #hacemos el centramiento
yd2p=1/(1+np.exp(-yd2c))#esta es la probabilidad de un score alto
XDn=np.array(XDB[['work_year','salary', 'salary_in_usd','company_size']])
XDn=XD/np.max(XD,axis=0) #normalizamos los datos
NE=4
model=Sequential()
model.add(Dense(10,activation='linear',use_bias=False,input_dim=NE))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='mse')
history=model.fit(XDn,yd2p,epochs=250,batch_size=581)
ys=model.predict(XDn)

#grafico de dispersion
plt.figure()
plt.scatter(yd2p,ys)
plt.show
#matriz de correlacion
df2=pd.DataFrame(np.column_stack((yd2p,ys)))
df2.corr()
print("la correlacion es:",df.corr())
#efectos independientes
W2=model.layers[0].get_weights()[0]
C2=model.layers[1].get_weights()[0]
np.dot(W2,C2)
print("los efectos independientes son:\n",np.dot(W2,C2))

#comoson las distribuciones de los datos
plt.figure()
sns.displot((yd2p.flatten(),ys.flatten()),kde=True)
plt.show()

#discrepancia entre medias
dscm=(np.mean(yd2p)-np.mean(ys)/np.mean(yd2p))
print("la discrepancia entre las medias es:",dscm) #si da negativo es que esta sobreentrando (estan por encima de los originales)

#discrepancia entre desviaciones
dscv=((np.std(yd2p)-np.std(ys))/np.std(yd2p))
print("la discrepancia entre dispersiones es:", dscv)

#la grafica de los datos de referencia es:
plt.figure()
plt.plot(yd2p[0:100,],'r',ys[0:100,],'b')
plt.show()

Probabilidad de que un individuo trabaje como head of data

In [ ]:
#Individuo: (2024,12000,87664,2)
XDi1=np.array([2024, 12000, 87664])
#Se normalizan las variables
XDi1n=XDi1/np.max(XD,axis=0)
print("Datos normalizados del individuo 1 \n",XDi1n)

In [ ]:
yi1=((0.90909091*-2.023058)+(0.6*-0.89059305)+(0.4*-1.6146948)+(0.11764706*-1.2491426)+(0.13333333*-1.7374382)+(0.8125*-0.11855859)+(0.49333333*10.533834))
yp1=1/(1+np.exp(-yi1))
print("La probabilidad del primer individuo para hacer parte de los ingresos altos es \n",yp1*100)